In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [10]:
week4_link = 'http://www.footballlocks.com/nfl_odds_week_4.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week4_html = requests.get(week4_link).text

soup = BeautifulSoup(week4_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[276])

<tr>
<td>9/27 8:20 ET</td>
<td>At LA Rams</td>
<td>-7</td>
<td>Minnesota</td>
<td>49</td>
<td>-$330 +$270</td>
</tr>
<tr>
<td>10/2 8:30 ET</td>
<td>At Philadelphia</td>
<td>-11.5</td>
<td>Green Bay</td>
<td>49.5</td>
<td>-$775 +$550</td>
</tr>


In [12]:
#want to 277 because it does not include last one
table = tr[51:277]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week4_df = pd.DataFrame(string_table)

week4_df.columns = ['all_data']

week4_df.head()

,all_data
0,<tr>\n<td>9/27 8:20 ET</td>\n<td>At LA Rams</t...
1,<tr>\n<td>9/30 1:00 ET</td>\n<td>At New Englan...
2,<tr>\n<td>9/30 1:00 ET</td>\n<td>Houston</td>\...
3,<tr>\n<td>9/30 1:00 ET</td>\n<td>At Atlanta</t...
4,<tr>\n<td>9/30 1:00 ET</td>\n<td>At Green Bay<...


In [14]:
#want to clean the date data

dates = [x.split('td')[1] for x in week4_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week4_df['date'] = cleaned_dates

In [20]:
#pull out favorites

week4_df['cleaned_favorites'] = week4_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week4_df['cleaned_underdogs'] = week4_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week4_df['cleaned_spread'] = week4_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week4_df['cleaned_over_under'] = week4_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week4_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week4_df['money_odds'] = money_odds

#input week number for each game
week4_df['week_number'] = '4'

In [21]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2018
year_list = []
for x in week4_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week4_df['year'] = year_list

In [22]:
#create a final dataframe for week 4
week4_final = week4_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week4_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,9/27 8:20 ET,2018,4,At LA Rams,Minnesota,-7,49,-$330 +$270
1,9/30 1:00 ET,2018,4,At New England,Miami,-6.5,50.5,-$310 +$250
2,9/30 1:00 ET,2018,4,Houston,At Indianapolis,-1,48.5,-$120 +$100
3,9/30 1:00 ET,2018,4,At Atlanta,Cincinnati,-3.5,52.5,-$190 +$165
4,9/30 1:00 ET,2018,4,At Green Bay,Buffalo,-9,44,-$400 +$310


In [23]:
#week4_final to pickle
week4_final.to_pickle('week4_final.pkl')